In [1]:
from helpers import data

In [2]:
rules, my_ticket, tickets = data(16, parser=lambda s: s.split("\n"), sep="\n\n")

In [3]:
tickets[:2], my_ticket

(['nearby tickets:',
  '279,705,188,357,892,488,741,247,572,176,760,306,410,861,507,906,179,501,808,245'],
 ['your ticket:',
  '73,101,67,97,149,53,89,113,79,131,71,127,137,61,139,103,83,107,109,59'])

Create one `tickets` list. 

In [4]:
# Remove text and combine
tickets.pop(0)
# Convert to lists of ints 
tickets = [[int(n) for n in row.split(",")] for row in tickets]

# Remove text 
my_ticket = [int(n) for n in my_ticket[1].split(",")] 

Create one `rules` dictionary. 

In [5]:
rules[:3]

['departure location: 26-715 or 727-972',
 'departure station: 45-164 or 175-960',
 'departure platform: 43-247 or 270-972']

In [6]:
def parse_rule(rule: str) -> tuple[str, list[tuple]]:
    """Parse rules into bounds.""" 
    name, rest = rule.split(":")
        
    bounds = []
    for bound in rest.strip().split(" or "):
        minimum, maximum = bound.split("-")
        bounds.append((int(minimum), int(maximum)))
    
    return (name, bounds)

assert parse_rule("departure location: 26-715 or 727-972") == ('departure location', [(26, 715), (727, 972)])

In [7]:
rules = dict(parse_rule(rule) for rule in rules)

In [8]:
rules

{'departure location': [(26, 715), (727, 972)],
 'departure station': [(45, 164), (175, 960)],
 'departure platform': [(43, 247), (270, 972)],
 'departure track': [(25, 306), (330, 949)],
 'departure date': [(26, 635), (660, 961)],
 'departure time': [(42, 773), (793, 961)],
 'arrival location': [(28, 928), (943, 952)],
 'arrival station': [(36, 593), (613, 966)],
 'arrival platform': [(33, 280), (297, 951)],
 'arrival track': [(44, 358), (371, 974)],
 'class': [(39, 815), (839, 955)],
 'duration': [(39, 573), (589, 959)],
 'price': [(49, 846), (865, 962)],
 'route': [(30, 913), (924, 954)],
 'row': [(29, 865), (890, 965)],
 'seat': [(44, 667), (683, 969)],
 'train': [(32, 473), (482, 969)],
 'type': [(40, 424), (432, 953)],
 'wagon': [(49, 156), (164, 960)],
 'zone': [(34, 521), (534, 971)]}

**Part 1:** Find the error rate, the sum of invalid values for any field. 

In [9]:
def within(val: int, minimum: int, maximum: int) -> bool:
    """Return whether val is within (minimum, maximum)."""
    return minimum <= val <= maximum

assert within(972, 26, 1715)
assert not within(3, 4, 100)

In [10]:
errors = [field for ticket in tickets 
                for field in ticket 
                if not any(within(field, *bound) 
                           for bounds in rules.values() 
                           for bound in bounds)]

In [11]:
sum(errors)

21081

**Part 1, Improved:** The main thing I need to clean up is converting a list of rule strings into a dictionary of name to ranges. My way of using `ChainMap` seems incomprehensible. Updated that above. 

**Part 1, Norvig:** Norvig created a class called `Sets` which inherits from `tuple` and is initialized with a list of tuples. It implements `in` which checks if an element is in any of the sets it holds. That's a pretty neat idea. 

In [12]:
class Sets(tuple):
    def __contains__(self, v):
        return any(within(v, *bound) for bound in self)
    
for k, v in rules.items():
    rules[k] = Sets(v)
    
assert 45 in rules["seat"]
assert 500 in rules["type"]
assert 1000 not in rules["row"]

errors = [field for ticket in tickets 
                for field in ticket 
                if not any(field in bounds for bounds in rules.values())]

sum(errors)

21081

**Part 2:** Discard the invalid tickets. Determine the order of the fields on each ticket, assuming the rest of the tickets are valid.  Multiply the six fields on my ticket that start with the word "departure".

My strategy is to create a mapping of index to set of possible fields. Then I'll iterate through that, on each step choose the index with fewest possible fields and assign it one, then remove that field from any other indices' possible fields, and repepat. 

In [13]:
tickets = [my_ticket, *tickets]
valid_tickets = [ticket 
                         for ticket in tickets
                         if all(any(field in bounds for bounds in rules.values()) for field in ticket)]

assert len(valid_tickets) == len(tickets) - len(errors)

In [14]:
possible_fields = {i: set(field_name 
                          for field_name, sets in rules.items()
                          if all(ticket[i] in sets
                                 for ticket in valid_tickets)) 
                   for i in range(len(valid_tickets[0]))}

field_assignments = dict()

In [15]:
while True:
    if not possible_fields:
        break 

    # Choose next index to assign 
    next_index = min(possible_fields, 
                     key=lambda k: len(possible_fields[k]))
    
    # Ensure it's still possible to find a matching 
    if not possible_fields[next_index]:
        raise ValueError(f"Finding a matching is impossible. No fields left for index {next_index}.")
    
    # Assign next_index a field name 
    assigned_field = possible_fields[next_index].pop()
    field_assignments[next_index] = assigned_field
    
    # Remove assigned field from all other possible_fields 
    for i in possible_fields:
        possible_fields[i] -= { assigned_field }
    
    # Remove next_index from possible_fields 
    del possible_fields[next_index]
    
field_assignments

{0: 'route',
 12: 'seat',
 4: 'arrival station',
 5: 'duration',
 1: 'arrival track',
 8: 'train',
 15: 'class',
 19: 'departure track',
 13: 'departure time',
 7: 'departure platform',
 16: 'departure date',
 14: 'departure station',
 2: 'departure location',
 17: 'price',
 11: 'wagon',
 18: 'zone',
 9: 'row',
 6: 'arrival platform',
 3: 'type',
 10: 'arrival location'}

In [16]:
prod = 1
for k, v in field_assignments.items():
    if "departure" in v:
        prod *= my_ticket[k]
        
prod

314360510573